In [20]:
import os
from langchain_openai import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SimpleSequentialChain, SequentialChain

os.environ["OPENAI_API_KEY"] = os.environ.get("OPENAI_API_KEY")

### For Single Input and Single Output Chain

In [2]:
llm = OpenAI(
    temperature = 0.6
)

In [3]:
restaurant_prompt = PromptTemplate(
    input_variables = ['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fancy name for my restaurant."
)

In [4]:
restaurant_prompt.format(cuisine = "Indian")

'I want to open a restaurant for Indian food. Suggest a fancy name for my restaurant.'

In [6]:
name_chain = LLMChain(
    llm = llm,
    prompt = restaurant_prompt
)
name_chain.run("Indo-Chinese")

'\n\n"Dragon Wok Fusion Bistro"'

In [7]:
menu_prompt = PromptTemplate(
    input_variables = ['restaurant_name'],
    template = "Suggest some menu items for {restaurant_name}. Return it as a comma separated list."
)

In [8]:
menu_chain = LLMChain(
    llm = llm,
    prompt = menu_prompt
)

In [13]:
chain = SimpleSequentialChain(
    chains = [name_chain, menu_chain]
)
response = chain.run("Brazilian")
print(response)



1. Feijoada (Brazilian bean stew)
2. Picanha (grilled beef sirloin)
3. Moqueca (Brazilian seafood stew)
4. Coxinha (fried chicken and cheese croquettes)
5. Acarajé (black-eyed pea fritters)
6. Churrasco (Brazilian-style BBQ)
7. Brigadeiros (chocolate truffles)
8. Caipirinha (Brazilian cocktail with cachaça, lime, and sugar)
9. Açaí bowl (blended açaí berries topped with granola and fruit)
10. Empadão (Brazilian savory pie with meat and vegetables)
11. Pão de queijo (cheese bread)
12. Bobó de camarão (shrimp in a creamy yucca sauce)
13. Moqueca de peixe (Brazilian fish stew)
14. Quindim (coconut custard dessert)
15. Pastel (fried pastry filled with meat or cheese)


### For Multiple inputs and Multiple Output Chains

In [14]:
llm = OpenAI(
    temperature = 0.6
)

In [15]:
restaurant_prompt = PromptTemplate(
    input_variables = ['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fancy name for my restaurant."
)

In [16]:
name_chain = LLMChain(
    llm = llm,
    prompt = restaurant_prompt,
    output_key = "restaurant_name"
)

In [18]:
menu_prompt = PromptTemplate(
    input_variables = ['restaurant_name'],
    template = "Suggest some menu items for {restaurant_name}. Return it as a comma separated list."
)

In [19]:
menu_chain = LLMChain(
    llm = llm,
    prompt = menu_prompt,
    output_key = "menu_items"
)

In [23]:
chain = SequentialChain(
    chains = [name_chain, menu_chain],
    input_variables = ['cuisine'],
    output_variables = ['restaurant_name', 'menu_items']
)

response = chain({'cuisine': 'Brazilian'})
print(response)

{'cuisine': 'Brazilian', 'restaurant_name': '\n\n"Sabor do Brasil" (Taste of Brazil)', 'menu_items': '\n\n1. Feijoada (Brazilian black bean stew)\n2. Moqueca (Brazilian seafood stew)\n3. Coxinha (Fried chicken croquette)\n4. Picanha (Brazilian grilled steak)\n5. Churrasco (Brazilian BBQ)\n6. Acarajé (Fried black-eyed pea fritters)\n7. Brigadeiro (Chocolate truffle)\n8. Caipirinha (Brazilian cocktail)\n9. Empadão (Savory pie with meat or vegetables)\n10. Açaí Bowl (Frozen açaí fruit topped with granola and fresh fruit)\n11. Pastel (Fried pastry filled with meat, cheese, or vegetables)\n12. Moqueca de Camarão (Brazilian shrimp stew)\n13. Farofa (Toasted cassava flour with seasonings)\n14. Quindim (Coconut custard dessert)\n15. Pão de Queijo (Cheese bread).'}
